<a href="https://colab.research.google.com/github/melbinjp/Deepcnn/blob/main/Copy_of_data_Loading_Training_Evaluating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code on (github.com/mr7495/covid19)

In [2]:
!nvidia-smi #show the allocated GPU

Wed May 25 13:58:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#Connect your Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#upgrade and install essential libraries
!pip install tensorflow
!pip install keras #--upgrade
!pip install zipfile36
!pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 462 kB 8.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 8.0 MB/s 


In [5]:
import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import pydicom as dicom
import zipfile
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal

In [6]:
# This is a link to kaggle dataset. If you have downloaded it already, save it as kaggle.zip in the current directory.
# If the link expired, get the new link from https://www.kaggle.com/c/rsna-pneumonia-detection-challenge/data

!wget -cO - 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/10338/862042/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1653744713&Signature=Durxt1xtU%2BV1RFrAtmckBuilzgugNMQPYVorE%2BDKAo%2BvjypiVJiTQP3GSVJZzRqnbU7ooGOxYZkHoaAUQjsukSJZjJs%2F%2BeyS1hbPy7bkIMls9aeKZRIKPhmdJfJIdZVECXJ4wt0wXQjE7IJJvUGc3onNALe1n%2BmyVaOw17%2FqCqDIc39OirpWLMTLjsrrVuwStiKSXkCgHqFdnF4vQUOHgsCDCXxKB378rDrWGqFDgZPKMVpzH9EHq9sekGIO0dtW%2BvHQXeGfPCTuPsiAGe3sD5qRKkGwarxtJDwQDe8mddlA5umV2336UWHUa0ou61WomkW9gJjetb1jDMq62DA4Qw%3D%3D&response-content-disposition=attachment%3B+filename%3Drsna-pneumonia-detection-challenge.zip' > kaggle.zip

--2022-05-25 13:59:07--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/10338/862042/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1653744713&Signature=Durxt1xtU%2BV1RFrAtmckBuilzgugNMQPYVorE%2BDKAo%2BvjypiVJiTQP3GSVJZzRqnbU7ooGOxYZkHoaAUQjsukSJZjJs%2F%2BeyS1hbPy7bkIMls9aeKZRIKPhmdJfJIdZVECXJ4wt0wXQjE7IJJvUGc3onNALe1n%2BmyVaOw17%2FqCqDIc39OirpWLMTLjsrrVuwStiKSXkCgHqFdnF4vQUOHgsCDCXxKB378rDrWGqFDgZPKMVpzH9EHq9sekGIO0dtW%2BvHQXeGfPCTuPsiAGe3sD5qRKkGwarxtJDwQDe8mddlA5umV2336UWHUa0ou61WomkW9gJjetb1jDMq62DA4Qw%3D%3D&response-content-disposition=attachment%3B+filename%3Drsna-pneumonia-detection-challenge.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3932287530 (3.7G) [application/zip]
Saving to: ‘STDOUT’

-     

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#Get the zip file I have shared, that contains the covid-chestxray-dataset images until 12 April
# Through the link below get the shared zip file and add it to your drive:
# https://drive.google.com/file/d/1Bwn4vTQUUB0tHK5aHh--Rk6eOxs2jg3q/view?usp=sharing
archive = zipfile.ZipFile('kaggle.zip') #Extract Kaggle Dataset
for file in archive.namelist():
     archive.extract(file, './All')
archive = zipfile.ZipFile('drive/My Drive/covid-chestxray-dataset.zip') #Extract covid-chestxray-Dataset
for file in archive.namelist():
     archive.extract(file, './covid-chestxray-dataset')

In [9]:
fold_num=1 #Select Fold Number

In [10]:
try:
  os.mkdir('All/All')
except:
  pass

In [11]:
!git clone https://github.com/mr7495/covid19 #connect to our repository on GitHub

Cloning into 'covid19'...
remote: Enumerating objects: 421, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 421 (delta 2), reused 0 (delta 0), pack-reused 411
Receiving objects: 100% (421/421), 6.56 MiB | 16.74 MiB/s, done.
Resolving deltas: 100% (223/223), done.


In [12]:
#Warning: Our prepared All.csv & train1.csv to train8.csv in each fold, are based on the covid-chestxray-dataset until 12 April.
#If you have used https://drive.google.com/file/d/1Bwn4vTQUUB0tHK5aHh--Rk6eOxs2jg3q/view?usp=sharing link to get the covid-chestxray-dataset.zip file(like the cells above), you can use our prepared csv files
#But if you want to load the updated covid-chestxray-dataset, you must make some changes to the csvfiles.
shutil.copy('covid19/prepared_csv_files/All.csv','All')
for i in range(1,9): #Load the 8 training phases csv files of the indicated fold
  shutil.copy('covid19/prepared_csv_files/fold{}/train{}.csv'.format(fold_num,i),'.')
  globals()['train{}'.format(i)]=[]

# The code for creating All.csv and training.csv files is available on  covid19/dataset preparing.ipynb.

In [13]:
images=[]
for r,d,f in os.walk('All/stage_2_train_images'): #Read the name of the images in both datasets
  for file in f:
    images.append(os.path.join(r,file))
for r,d,f in os.walk('covid-chestxray-dataset/images'):
  for file in f:
   images.append(os.path.join(r,file))

In [14]:
csv_all=pd.read_csv('All/All.csv', nrows=None) #Read the CSV file that contains the names of the images with their labels.
for index, row in csv_all.iterrows(): #This loop reads the images, converts them to suitable format and saves them in the All directory
  if '.png' in row['filename']: #For creating the All.csv we have converted the kaggle dataset images to png format,
                                #but some of the images in the other dataset also are in the format of png, so we use try/except here to distinguish which dataset, the annotation in the CSV file belongs to.
    try:
      png_index=row['filename'].find('.png')
      last_name=row['filename'][:png_index]+'.dcm'
      ds = dicom.dcmread(os.path.join('All/stage_2_train_images',last_name))
      pixel_array_numpy = ds.pixel_array
      imgname = last_name[:-4]+'.png'
      cv2.imwrite(os.path.join('All/All', imgname), pixel_array_numpy)
    except:
      png_index=row['filename'].find('.png')
      img=cv2.imread(os.path.join('covid-chestxray-dataset/images',row['filename'][:png_index+4])) 
      gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
      cv2.imwrite(os.path.join('All/All', row['filename'][:png_index+4]), gray)  
  else:
    img=cv2.imread(os.path.join('covid-chestxray-dataset/images',row['filename']))
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    cv2.imwrite(os.path.join('All/All', row['filename']), gray)  

In [15]:
All=[] #Thie list that is readed from All.csv
all_train=[] #This list contains the training annotations
all_test=[]
with open('All/All.csv',newline='', mode='r') as csvfile: #Adding All.csv rows to All list
      csvreader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      for row in csvreader:
          All.append(row)
for i in range(1,9): #Adding training1.csv to training8.csv rows to All_train list. This 1 to 8 indicate the 8 training phases
  with open('train{}.csv'.format(i),newline='', mode='r') as csvfile:
      csvreader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      for row in csvreader:
        all_train.append(row)
with open('all_test.csv'.format(i),newline='', mode='w') as csvfile: #Add all the other images that do not belong to the training phases, to the test set
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['filename','class'])
    for row in All:
      if row not in all_train:
        csvwriter.writerow(row)

In [16]:
#Because we have written our code somehow to only save the epochs with the best validation accuracy during the training,
# we created the s_test.csv with 631 images. That is why validating each epoch for 11302 images during training would be terribly time-consuming
#so we select a random s_test.csv for evaluating the network during the training process.

In [17]:
for i in range(10): #Shuffle the All list
  random.shuffle(All)
with open('s_test.csv'.format(i),newline='', mode='w') as csvfile: #Create s_test.csv file for evaluating the network during training
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['filename','class'])
    ln=0
    lp=0
    for row in All:
      if row not in all_train:
        if row[1]=='COVID-19':
          csvwriter.writerow(row)
        elif row[1]=='normal':
          if ln<300:
            csvwriter.writerow(row)
            ln+=1
        else:
          if lp<300:
            csvwriter.writerow(row)
            lp+=1

# New Section

In [18]:
#remove the unnecessary file to increase the free space
try:
  os.remove('kaggle.zip')
  shutil.rmtree('All/stage_2_train_images')
  shutil.rmtree('All/stage_2_test_images')
  shutil.rmtree('covid-chestxray-dataset')
except:
  pass

In [19]:
#Here we set the data generators for applying data augmentation methods
train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rescale=1./255,zoom_range=0.05,rotation_range=360,width_shift_range=0.05,height_shift_range=0.05,shear_range=0.05)
test_datagen = ImageDataGenerator(rescale=1./255)
test_df = pd.read_csv("s_test.csv")

In [20]:
# Now our data is ready
# We create the neural network in the next level
#If you want to use our trained network you can visit this shared folde:
#  https://drive.google.com/drive/folders/19R4T-D-bWUkQOh3xy5CkIDAmkLBt8ID7?usp=sharing

In [21]:
 mod_attrs_str = dir(keras.optimizers)
 print(mod_attrs_str)

['Optimizer', 'TFOptimizer', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'adadelta_experimental', 'adadelta_v2', 'adagrad_experimental', 'adagrad_v2', 'adam_experimental', 'adam_v2', 'adamax_v2', 'backend', 'deserialize', 'deserialize_keras_object', 'ftrl', 'get', 'gradient_descent_v2', 'keras_export', 'nadam_v2', 'optimizer_experimental', 'optimizer_v2', 'rmsprop_v2', 'serialize', 'serialize_keras_object', 'sgd_experimental', 'tf']


In [ ]:
import keras.backend as k
k.clear_session() #Clear keras backend 
try:
  os.mkdir('models')
except:
  pass
full_name='concatenate'
classes_number=3 #Number of classes
input_tensor=Input(shape=(300,300,3))
######################################################################################################
base_model1 = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)
features1 = base_model1.output
######################################################################################################
base_model2 = ResNet50V2(weights='imagenet', include_top=False, input_tensor=input_tensor)
features2 = base_model2.output
concatenated=keras.layers.concatenate([features1,features2]) #Concatenate the extracted features
####################################################################################################
conv=keras.layers.Conv2D(1024, (1, 1),padding='same')(concatenated) #add the concatenated features to a convolutional layer
feature = Flatten(name='flatten')(conv)
dp = Dropout(0.5)(feature) #add dropout
preds = Dense(classes_number, activation='softmax', kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp) 
Concatenated_model = Model(inputs=input_tensor, outputs=preds)
#######################################################
for layer in Concatenated_model.layers:
  layer.trainable = True
Concatenated_model.compile(optimizer=optimizers.nadam_v2.Nadam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])
filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%full_name 
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max') #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]

for epoch_num in range(1,9): # This loop applies the 8 phases for training
  train_df =pd.read_csv("train{}.csv".format(epoch_num)) #Add data generators for parsing images to the network while training
  train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='All/All',
        x_col="filename",
        y_col="class",
        target_size=(300, 300),
        batch_size=20,
        class_mode='categorical',shuffle=True)
  validation_generator = test_datagen.flow_from_dataframe(
          dataframe=test_df,
          directory='All/All',
          x_col="filename",
          y_col="class",
          target_size=(300, 300),
          batch_size=20,
          class_mode='categorical',shuffle=True)
  Concatenated_model.fit_generator(train_generator, epochs=100,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training



94683136/94668760 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


Found 633 validated image filenames belonging to 3 classes.
Found 631 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/100
32/32 [==============================] - 96s 2s/step - loss: 0.8898 - accuracy: 0.6161 - val_loss: 0.6087 - val_accuracy: 0.7559
Epoch 2/100
32/32 [==============================] - 46s 1s/step - loss: 0.6791 - accuracy: 0.7346 - val_loss: 0.6297 - val_accuracy: 0.7971
Epoch 3/100
32/32 [==============================] - 43s 1s/step - loss: 0.5896 - accuracy: 0.7915 - val_loss: 0.8734 - val_accuracy: 0.6624
Epoch 4/100
32/32 [==============================] - 46s 1s/step - loss: 0.5226 - accuracy: 0.8073 - val_loss: 0.3890 - val_accuracy: 0.8669
Epoch 5/100
32/32 [==============================] - 43s 1s/step - loss: 0.4581 - accuracy: 0.8294 - val_loss: 0.5112 - val_accuracy: 0.8352
Epoch 6/100
32/32 [==============================] - 44s 1s/step - loss: 0.2887 - accuracy: 0.8926 - val_loss: 0.4263 - val_accuracy: 0.8510
Epoch 7/100
32/32 [==============================] - 44s 1s/step - loss: 0.3719 - accuracy: 0.8641 - val_loss: 0.4524 - val_accuracy: 0.8637
Epoch 8/100
3

In [ ]:
#After the training is done, we attempt to evaluate our saved networks.

In [ ]:
trained_models=[]
for r,d,f in os.walk('models'):
  for file in f:
    trained_models.append(os.path.join(r,file)) #Add the saved models in the models directory
reports={}
test_df = pd.read_csv("all_test.csv") #Load the full test CSV file that includes 11302 images
validation_generator = test_datagen.flow_from_dataframe( #Set new data generator
        dataframe=test_df,
        directory='All/All',
        x_col="filename",
        y_col="class",
        target_size=(300, 300),
        batch_size=20,
        class_mode='categorical',shuffle=True)
for trained_model in trained_models:
  k.clear_session()
  net=keras.models.load_model(trained_model) #load model
  covid_label= validation_generator.class_indices['COVID-19'] #get the index of COVUD-19 class 
  pneu_label= validation_generator.class_indices['pneumonia'] #get the index of pneumonia class 
  normal_label= validation_generator.class_indices['normal']  #get the index of normal class 
  tp=0 #True Positives
  fp=0 #False Positives
  anum=0 #All the images numbers
  ###########
  wrong_covid=0
  correct_covid=0
  not_detected_covid=0
  covid_num=0
  ###########
  wrong_pneu=0
  correct_pneu=0
  not_detected_pneu=0
  pneu_num=0
  ############
  wrong_normal=0
  correct_normal=0
  not_detected_normal=0
  normal_num=0
  ##############
  wrong_covid_normal=0 #COVID-19 class which has been detected as normal, incorrectly
  wrong_covid_pneu=0   #COVID-19 class which has been detected as pneumonia, incorrectly
  wrong_pneu_covid=0   #pneumonia class which has been detected as COVID-19 , incorrectly
  wrong_pneu_normal=0  #pneumonia class which has been detected as normal , incorrectly
  wrong_normal_pneu=0  #normal class which has been detected as pneumonia , incorrectly
  wrong_normal_covid=0  #normal class which has been detected as COVID-19 , incorrectly
  ################
  for num,img_name in enumerate(validation_generator.filenames): #load image
    gt_ind=validation_generator.classes[num] #get the loaded image class index
    img=cv2.resize(cv2.imread(os.path.join('All','All',img_name)),(300,300)) #resize image
    img=img.astype('float32') / 255.0 #scale the image
    pred_ind=np.argmax(net.predict(np.expand_dims(img,axis=0))[0]) #get the predicted class index

    anum+=1 #count the number of images


    if gt_ind==covid_label:
      covid_num+=1
    if gt_ind==pneu_label:
      pneu_num+=1
    if gt_ind==normal_label:
      normal_num+=1
    ##################
    if gt_ind==covid_label and pred_ind==covid_label: 
      correct_covid+=1
    if gt_ind==covid_label and pred_ind!=covid_label:
      not_detected_covid+=1
      if pred_ind==pneu_label:
        wrong_covid_pneu+=1
      elif pred_ind==normal_label:
        wrong_covid_normal+=1
    if gt_ind!=covid_label and pred_ind==covid_label:
      wrong_covid+=1
    ###########################################
    if gt_ind==normal_label and pred_ind==normal_label: 
      correct_normal+=1
    if gt_ind==normal_label and pred_ind!=normal_label:
      not_detected_normal+=1
      if pred_ind==pneu_label:
        wrong_normal_pneu+=1
      elif pred_ind==covid_label:
        wrong_normal_covid+=1
    if gt_ind!=normal_label and pred_ind==normal_label:
      wrong_normal+=1
    ###########################################
    if gt_ind==pneu_label and pred_ind==pneu_label: 
      correct_pneu+=1
    if gt_ind==pneu_label and pred_ind!=pneu_label:
      not_detected_pneu+=1
      if pred_ind==normal_label:
        wrong_pneu_normal+=1
      elif pred_ind==covid_label:
        wrong_pneu_covid+=1
    if gt_ind!=pneu_label and pred_ind==pneu_label:
      wrong_pneu+=1
    ###########################################

    if pred_ind==gt_ind:
      tp+=1
    else:
      fp+=1

  reports[trained_model[7:]]=[correct_covid,wrong_covid,not_detected_covid,
                   correct_pneu,wrong_pneu,not_detected_pneu,
                   correct_normal,wrong_normal,not_detected_normal,
                   covid_num,pneu_num,normal_num,tp,fp,wrong_covid_normal,wrong_covid_pneu,
                   wrong_pneu_covid,wrong_pneu_normal,wrong_normal_pneu,wrong_normal_covid]


  print(trained_model[7:])
  print('correct_covid: ',correct_covid/covid_num,'%')
  print('wrong_covid: ',wrong_covid)
  print('not_detected_covid: ',not_detected_covid/covid_num,'%')
  print('###########################')

with open('{}-fold{}.csv'.format(trained_model[7:],fold_num),newline='', mode='w') as csvfile: #save the evaluation data into csvfile
     csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
     csv_writer.writerow(['name','correct_covid','wrong_covid','not_detected_covid',
                          'correct_pneu','wrong_pneu','not_detected_pneu',
                          'correct_normal','wrong_normal','not_detected_normal',
                          'covid_num','pneu_num','normal_num','tp','fp','wrong_covid_normal','wrong_covid_pneu',
                  'wrong_pneu_covid','wrong_pneu_normal','wrong_normal_pneu','wrong_normal_covid'])
     for key in reports.keys():
         row=[key]
         row.extend(reports[key])